In [1]:
import cv2
import torch
import json
import pandas as pd
from PIL import Image
from tqdm import tqdm

In [2]:
import torch
# exp_name 변경해서 돌리면 됩니다
exp_name = 'exp'
model = torch.hub.load('./utils', 'custom', path= f'../output/yolov5/{exp_name}/weights/best.pt', source='local') 
model.conf = 0.001  # confidence threshold (0-1)
model.iou = 0.6  # NMS IoU threshold (0-1)

prediction_string = ['']  * 4871 
image_id = [f'test/{i:04}.jpg' for i in range(4871)]
for i in tqdm(range(4871)):
    img = Image.open(f'../dataset/test/{i:04}.jpg')

    results = model(img, size=1024, augment=True) # augment로 TTA on/off 가능합니다. 
    for bbox in results.pandas().xyxy[0].values:
        xmin, ymin, xmax, ymax, confidence, clss, name = bbox
        prediction_string[i] += f'{clss} {confidence} {xmin} {ymin} {xmax} {ymax} '


YOLOv5 🚀 b7f9639 torch 1.7.1 CUDA:0 (Tesla V100-PCIE-32GB, 32510.5MB)

Fusing layers... 
Model Summary: 574 layers, 140057380 parameters, 0 gradients, 208.3 GFLOPs
Adding AutoShape... 
100%|██████████| 4871/4871 [12:13<00:00,  6.65it/s]


In [3]:
raw_data ={
    'PredictionString' : prediction_string,
    'image_id' : image_id
}
dataframe = pd.DataFrame(raw_data)

# output/yolov5/exp_name에 저장됩니다.
dataframe.to_csv(f'../output/yolov5/{exp_name}/submission_{exp_name}.csv', sep=',', na_rep='NaN', index=None)